In [2]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from dotenv import load_dotenv
import time
from tqdm.notebook import tqdm
import random
from io import StringIO

# Define environment variables for username and password
load_dotenv()
username = os.getenv('CHAMPIONDATA_USERNAME')
password = os.getenv('CHAMPIONDATA_PASSWORD')
JSESSIONID = 'node0q9bpwcnsm724lhmnjb3wwcq933470.node0'
SESSION_ID = 'CIA_1726697120856' # set manually for now

In [3]:
# Function to login and retrieve session cookies
# TODO: login headless, fetch jsessionid, session_id
def login_and_get_session(session_id=SESSION_ID, jsession_id=JSESSIONID, login_url="https://recruit.championdata.com/login.jsp"):
    session = requests.Session()

    # Get login page to retrieve necessary hidden form fields (if any)
    # login_page = session.get(login_url)
    # soup = BeautifulSoup(login_page.text, 'html.parser')

    # Prepare login data (add additional hidden fields if necessary)
#     login_data = {
#         'username': username,
#         'password': password
#     }
    
#     response = session.post(login_url, data=login_data)
    
    #if response.status_code == 200 and "Welcome" in response.text:
    # Print the cookies attached to the session
    print("Cookies after login:")
    for cookie in session.cookies:
        print(f"{cookie.name}: {cookie.value}")

    # Capture session_id if present
    # session_id = session.cookies.get('session_id')

    # Manually add the session_id if it is not present
    if not any(cookie.name == 'session_id' for cookie in session.cookies):
        session.cookies.set('session_id', session_id)  # Replace with the actual session_id value from your browser

    if not any(cookie.name == 'session_id' for cookie in session.cookies):
        session.cookies.set('JSESSIONID', jsession_id)

    return session
    #else:
    #    raise Exception("Login failed")

# TODO: centralise somewhere
competitions_dict = {
    1: {"Name":"AFL","Start":2011},
    264: {"Name":"AFL Women's","Start":2017},
    414: {"Name":"AFLW U18CH","Start":2017},
    264: {"Name":"AFLW Academy","Start":2022},
    724 : {"Name":"AFLW U17CH","Start":2021},
    # 394 : {"Name":"AFL U18 Academy","Start":2017},
    3 : {"Name":"AFL AIS Academy","Start":2014},
    # 136 : {"Name":"NEAFL","Start":2014},
    16: {"Name":"SANFL League","Start":2014},
    133: {"Name":"SANFL U18","Start":2014},
    123: {"Name":"SANFL Reserve","Start":2014},
    144: {"Name":"SANFL U16","Start":2014},
    364: {"Name":"SANFL Women's","Start":2016},
    127: {"Name":"School Boys","Start":2014},
    143: {"Name":"STATE","Start":2014},
    11: {"Name":"NAB League","Start":2014},
    574: {"Name":"NAB League Girls","Start":2019},
    126: {"Name":"TRIAL","Start":2019},
    #135: {"Name":"TSL","Start":2011},
    12: {"Name":"U16 Championships","Start":2014},
    # 315: {"Name":"U17 Championships","Start":2016},
    19: {"Name":"U18 Championships","Start":2014},
    15:{"Name":"VFL","Start":2014},
    464:{"Name":"VFLW","Start":2018},
    120:{"Name":"WAFL League","Start":2014},
    122:{"Name":"WAFL Colts","Start":2014},
    121:{"Name":"WAFL Reserve","Start":2014},
    764:{"Name":"WAFL Womens","Start":2022},
    # 484: {"Name":"Women's Trial Matches","Start":2022}
}

dimensional_custom_selections =("MBM_D50_BOUNDARY!-!MBM_D50_BOUNDARY_PCT!-!MBM_D50_CORRIDOR!-!MBM_D50_CORRIDOR_PCT"
"!-!D50_REB_IN50!-!D50_REB_IN50_PCT!-!LONG_DOWN_LINE_KICK!-!LONG_DOWN_LINE_LOSE!-!LONG_DOWN_LINE_LOSE_PCT!-!"
"LONG_DOWN_LINE_OPP!-!LONG_DOWN_LINE_PCT!-!LONG_DOWN_LINE_RET!-!LONG_DOWN_LINE_RET_PCT!-!LONG_DOWN_LINE_SCORE!-!"
"LONG_DOWN_LINE_SCORE_PCT!-!LONG_DOWN_LINE_STOP!-!LONG_DOWN_LINE_STOP_PCT!-!MID_BALL_MOVE_BOUND!-!MID_BALL_MOVE_BOUND_PCT!-!"
"MID_BALL_MOVE_CORRIDOR!-!MID_BALL_MOVE_CORRIDOR_PCT!-!MBM_D50!-!MID_BALL_MOVE_WING!-!MID_BALL_MOVE_WING_PCT!-!"
"MBM_D50_WING!-!MBM_D50_WING_PCT!-!*DISPOSAL!-!DISPOSAL_EFFICIENCY!-!EFFECTIVE_DISPOSAL!-!EFFECTIVE_HANDBALL!-!"
"EFFECTIVE_KICK!-!HANDBALL!-!HANDBALL_EFFICIENCY!-!KICK!-!KICK_TO_HANDBALL_RATIO!-!KICK_EFFICIENCY!-!*"
"DISPOSAL_POST!-!DISPOSAL_PRE!-!KICK_FWD_PCT!-!KICK_FWD_GEN_PCT!-!KICK_FWD_SET_PCT!-!KICK_LONG_GEN_PCT!-!*"
"KICK_LONG_SET_PCT!-!LONG_KICK_FROM_POSS_PCT!-!*50M_PENALTY!-!FREE_AGAINST!-!FREE_AGAINST_GENERAL!-!FREE_AGAINST_MARKING"
"!-!FREE_AGAINST_RUCK!-!FREE_AGAINST_TACKLING!-!FREE_FOR!-!FREE_FOR_POST!-!FREE_FOR_PRE!-!M50M_PENALTY_AGAINST!-!*"
"CLANGER_KICK!-!SHORT_KICK!-!LONG_KICK!-!GROUND_KICK!-!INEFFECTIVE_KICK!-!KICK_BACKWARDS!-!LONG_EFF_KICK_FROM_POSS!-!"
"LONG_KICK_FROM_POSS!-!LONG_KICK_FROM_POS_EFF_PCT!-!OUT_ON_FULL!-!SHORT_EFF_KICK_FROM_POSS!-!SHORT_KICK_FROM_POSS!-!"
"SHRT_KICK_FROM_POS_EFF_PCT!-!SHORT_KICK_FROM_POSS_PCT!-!*KI_IN50_LAUNCH!-!KICK_IN!-!KICK_IN_CORRIDOR!-!KICK_IN_CORRIDOR_PCT!-!"
"KICK_IN_EFFECTIVE!-!KICK_IN_EFFECTIVE_PCT!-!KICK_IN_LEFT!-!KICK_IN_LEFT_PCT!-!KICK_IN_RIGHT!-!KICK_IN_RIGHT_PCT!-!"
"KICK_IN_SELF!-!KICK_IN_SELF_PCT!-!KI_PROG_MID!-!KI_RES_CP_LOSE!-!KI_RES_CP_LOSE_PCT!-!KI_RES_CP_RET!-!KI_RES_CP_RET_PCT!-!"
"KI_RES_LOSE!-!KI_RES_LOSE_PCT!-!KI_RES_OPP_SCORE!-!KI_RES_RET!-!KI_RES_RET_PCT!-!KI_RES_SCORE!-!KI_RES_STOP!-!"
"KI_RES_STOP_PCT!-!KI_RES_UP_LOSE!-!KI_RES_UP_LOSE_PCT!-!KI_RES_UP_RET!-!KI_RES_UP_RET_PCT!-!KI_SCORE_LAUNCH!-!*"
"CONTESTED_MARK_FROM_OPP!-!CONTESTED_MARK_FROM_TEAM!-!MARK!-!MARK_DROPPED!-!MARK_FROM_OPP_KICK!-!MARK_FUMBLED!-!"
"MARK_ON_LEAD!-!MARK_PLAY_ON!-!MARK_PLAY_ON_PCT!-!UNCONTESTED_MARK_FROM_OPP!-!UNCONTESTED_MARK_FROM_TEAM!-!*"
"HANDBALL_GAIN_METRES!-!HB_GAIN_METRES_PER_HB!-!KICK_GAIN_METRES!-!KICK_GAIN_METRES_PER_KICK!-!TOTAL_GAINED_METRES!-!"
"TOTAL_GAIN_METRES_PER_DISP!-!*CHAIN_METRES!-!CHAIN_METRES_BU!-!CHAIN_METRES_CB!-!CHAIN_METRES_KI!-!CHAIN_METRES_NET!-!"
"CHAIN_METRES_NET_BU!-!CHAIN_METRES_NET_CB!-!CHAIN_METRES_NET_KI!-!CHAIN_METRES_NET_PG!-!CHAIN_METRES_NET_ST!-!"
"CHAIN_METRES_NET_TI!-!CHAIN_METRES_PG!-!CHAIN_METRES_ST!-!CHAIN_METRES_TI!-!METRES_GAINED_ASSISTED!-!METRES_GAINED_EFF!-!"
"METRES_GAINED_NET!-!METRES_GAINED_RETAINED!-!*CONTEST_DEF_LOSE!-!CONTEST_DEF_LOSE_PCT!-!CONTEST_DEF_MARK_LOSE!-!"
"CONTEST_DEF_MARK_LOSE_PCT!-!CONTEST_DEF_MARK_WIN!-!CONTEST_DEF_MARK_WIN_PCT!-!CONTEST_DEF_NEUTRAL!-!"
"CONTEST_DEF_NEUTRAL_PCT!-!CONTEST_DEF_ONE_ON_ONE!-!CONTEST_DEF_WIN!-!CONTEST_DEF_WIN_PCT!-!CONTEST_TARG_LOSE!-!"
"CONTEST_TARG_LOSE_PCT!-!CONTEST_TARG_MARK_LOSE!-!CONTEST_TARG_MARK_LOSE_PCT!-!CONTEST_TARG_MARK_WIN!-!"
"CONTEST_TARG_MARK_WIN_PCT!-!CONTEST_TARG_NEUTRAL!-!CONTEST_TARG_NEUTRAL_PCT!-!CONTEST_TARG_ONE_ON_ONE!-!"
"CONTEST_TARG_WIN!-!CONTEST_TARG_WIN_PCT!-!CONTEST_DEFENDER!-!CONTEST_TARGET!-!*BAULK!-!BAULKED!-!BLOCK!-!BROKEN_TACKLE"
"!-!KNOCK_ON!-!MISSED_TACKLE!-!RUNDOWN_TACKLE!-!RUNNING_BOUNCE!-!SMOTHER!-!SPOIL!-!SPOIL_EFFICIENCY!-!"
"SPOIL_GAINING_POSSESSION!-!SPOIL_INEFFECTIVE!-!TACKLE!-!TACKLE_POST!-!TACKLE_PRE!-!*RATING!-!RATING_50M_PENALTY!-!"
"RATING_BALL_USE!-!RATING_BALL_WINNING!-!RATING_DEBITS!-!RATING_DEFENCE!-!RATING_FIELD_KICKS!-!RATING_FREE_AGAINST!-!"
"RATING_HANDBALL!-!RATING_HITOUTS!-!RATING_KICK_IN!-!RATING_NEGATIVES!-!RATING_POSS_INTERCEPT!-!RATING_POSS_MID_CHAIN!-!"
"RATING_POSS_STOPPAGE!-!RATING_PRESSURE!-!RATING_SHOTS!-!RATING_SPOILS!-!*CONTESTED_KNOCK_ON!-!CONTESTED_MARK!-!"
"CONTESTED_POSSESSION!-!CONTESTED_POSSESSION_POST!-!CONTESTED_POSSESSION_PRE!-!CON_POSSESSION_RATE!-!CRUMB!-!"
"GATHER!-!GATHER_FROM_HITOUT!-!GROUND_BALL_GET!-!GROUND_BALL_GET_POST!-!GROUND_BALL_GET_PRE!-!RECEIVE_HANDBALL!-!"
"HARD_BALL_GET!-!HARD_BALL_GET_POST!-!HARD_BALL_GET_PRE!-!LOOSE_BALL_GET!-!LOOSE_BALL_GET_POST!-!LOOSE_BALL_GET_PRE!-!"
"RUCK_HARD_BALL_GET!-!UNCONTESTED_MARK!-!UNCONTESTED_POSSESSION!-!UNC_POSSESSION_RATE!-!*FREE_AGAINST_TACKLING!-!"
"PLY_ATTEMPTED_TACKLE!-!PLY_PRES_IMP_CHASING!-!PLY_PRES_IMP_CLOSING!-!PLY_PRES_IMP_CORRALLING!-!PLY_INEFFECTIVE_TACKLE!-!"
"PLAYER_PRESSURE_ACT!-!PLY_PRES_PHYSICAL!-!PLY_PRESS_PTS!-!PLY_TACKLE_EFFICIENCY!-!SQUAD_ATTEMPTED_TACKLE!-!SQUAD_IMP_CHASING!-!"
"SQUAD_IMP_CLOSING!-!SQUAD_IMP_CORRALLING!-!SQUAD_INEFFECTIVE_TACKLE!-!SQUAD_PRESSURE_CHANCE!-!SQUAD_PRESSURE_FACTOR!-!"
"SQUAD_PRESSURE_POINTS!-!SQUAD_TACKLE_EFFICIENCY!-!TURNOVER_FORCED!-!TURNOVER_FORCED_BEHI_RUSH!-!TURNOVER_FORCED_GOAL!-!"
"TURNOVER_FORCED_PTS!-!TURNOVER_FORCED_SCORE_PCT!-!*DREAMTEAM_PTS!-!RANKING_PTS!-!*INT_CAP_TOTAL!-!INT_OFF_BLOOD!-!"
"INT_OFF_CONCUSSION!-!INT_OFF_INJURED!-!INT_OFF_NORMAL!-!INT_OFF_OUT!-!INT_OFF_TOTAL!-!OFF_STINT_SECS!-!"
"ON_STINT_SECS!-!PERIOD_LENGTH!-!SUB_OFF!-!SUB_ON!-!TIME_ON_GROUND_PCT!-!TOG_PLAYED_SECS!-!*BEHIND!-!BEHIND_ASSIST!-!"
"CLUB_SCORE_INVOLVEMENT_PCT!-!EXPECTED_ACCURACY!-!EXPECTED_GOAL!-!EXPECTED_SCORE!-!EXPECTED_SCORE_DIFF!-!GOAL!-!"
"GOAL_ASSIST!-!KI_SCORING_POINTS!-!MISSED_SHOT!-!SQUAD_POINTS!-!RUSHED_BEHIND!-!SCORE_ACCURACY_PCT!-!SCORE_ASSIST!-!"
"SCORE_INVOLVEMENT!-!SCORE_INVOLVEMENT_PCT!-!SCORE_LAUNCH!-!SCOREBOARD_IMPACT!-!SCORING_SHOTS!-!SHOT_AT_GOAL!-!"
"SHOT_AT_GOAL_PCT!-!ST_SCORING_POINTS!-!TO_SCORING_POINTS!-!*ADVANTAGE_RATE!-!BALL_UP!-!BALL_UP_CLEARANCE!-!"
"BALL_UP_FIRST_POSSESSION!-!BALL_UP_HITOUT!-!BALL_UP_HITOUT_SHARKED!-!BU_HITOUT_TO_ADVANTAGE!-!BU_SCORE_LAUNCH!-!"
"BALL_UP_VERSUS!-!CENTRE_BOUNCE!-!CB_ATTEND!-!CENTRE_BOUNCE_CLEARANCE!-!CB_FIRST_POSSESSION!-!CENTRE_BOUNCE_HITOUT!-!"
"CB_HITOUT_SHARKED!-!CB_HITOUT_TO_ADVANTAGE!-!CENTRE_BOUNCE_MID!-!CB_SCORE_LAUNCH!-!CENTRE_BOUNCE_VERSUS!-!CLEARANCE!-!"
"CLEARANCE_WIN_PCT!-!FIRST_POSSESSION!-!FP_CLEAR_PCT!-!FP_WIN_PCT!-!HIT_OUT_SHARKED!-!HIT_OUT_TO_ADVANTAGE!-!HITOUT!-!"
"HITOUT_TO_ADV_WIN_PCT!-!HITOUT_WIN_PCT!-!HITOUT_WIN_PCT_PLY!-!RUCK_CONTEST!-!SEC_STOP!-!STOPPAGE!-!THROW_IN!-!"
"THROW_IN_CLEARANCE!-!THROW_IN_FIRST_POSSESSION!-!THROW_IN_HITOUT!-!THROW_IN_HITOUT_SHARKED!-!TI_HITOUT_TO_ADVANTAGE!-!"
"TI_SCORE_LAUNCH!-!THROW_IN_VERSUS!-!*BU_CHAIN_LAUNCH!-!BALL_UP_HITOUT_SHARK!-!CB_ABSENT_CLEAR_DIFF!-!CB_ABSENT_TEAM_CLR_WIN_PCT!-!"
"CB_ABSENT!-!CB_ATTEND_CLEAR_DIFF!-!CB_ATTEND_TEAM_CLR_WIN_PCT!-!CB_BEHIND!-!CB_CHAIN_LAUNCH!-!CB_GOAL!-!"
"CENTRE_BOUNCE_HITOUT_SHARK!-!CB_RUSHED_BEHIND!-!EFFECTIVE_CLEARANCE!-!EFF_CLR_PCT!-!EFF_CLR_SCORE_PCT!-!"
"HIT_OUT_SHARK!-!HOLD!-!BU_IN50_LAUNCH!-!CB_IN50_LAUNCH!-!TI_IN50_LAUNCH!-!STOP_EXIT_H!-!STOP_EXIT_H_PCT!-!"
"STOP_EXIT_K!-!STOP_EXIT_K_PCT!-!THIRD_MAN_UP!-!TI_CHAIN_LAUNCH!-!THROW_IN_HITOUT_SHARK!-!TIME_IN_POSS_DIFF!-!"
"TIME_IN_POSS_DIFF_PCT!-!TIME_IN_POSS_DISPUTE!-!TIME_IN_POSS_DISPUTE_PCT!-!TIME_IN_POSS_OPP_SQUAD_PCT!-!"
"TIME_IN_POSS_OPP_SQUAD!-!TIME_IN_POSS_SQUAD!-!TIME_IN_POSS_SQUAD_PCT!-!TIME_IN_ZONE_PCT!-!TIME_IN_ZONE_SECS!-!"
"*DISPOSALS_PER_TURNOVER!-!INTERCEPT!-!TURNOVER!-!*PG_IN50_LAUNCH!-!INTERCEPT_BEHI_LAUNCH!-!INTERCEPT_GOAL_LAUNCH!-!"
"TURNOVER_FORCED_GIVEAWAY!-!TURNOVER_GIVEAWAY!-!TURNOVER_UNFORCED_GIVEAWAY!-!*D50_REBOUND!-!INSIDE_50!-!IN50_KICK!-!"
"IN50_KICK_MARK!-!IN50_KICK_MARK_PCT!-!IN50_KICK_RETAIN!-!IN50_KICK_RETAIN_PCT!-!REBOUND_50!-!RE50_RESULT_OPP_CP_PCT!-!"
"RE50_RESULT_OPP_UP_PCT!-!RE50_RESULT_OPP_CP!-!RE50_RESULT_OPP_UP!-!RE50_RESULT_OTHER!-!RE50_RESULT_OTHER_PCT!-!"
"RE50_RESULT_RUN!-!RE50_RESULT_RUN_PCT!-!RE50_RESULT_STOPPAGE!-!RE50_RESULT_STOPPAGE_PCT!-!RE50_RESULT_TEAM_CP!-!"
"RE50_RESULT_TEAM_CP_PCT!-!RE50_RESULT_TEAM_UP_PCT!-!RE50_RESULT_TEAM_UP!-!SQUAD_GOALW_PER_IN50!-!SQUAD_SCORE_PER_IN50!-!"
"*CHAIN_LAUNCH!-!IN50_FORCED!-!IN50_FORCED_RETURN!-!IN50_INIT!-!IN50_INIT_DIR!-!IN50_INIT_DIR_RETURN!-!"
"IN50_INIT_RETURN!-!IN50_KICK_BOMB!-!IN50_KICK_GOTO!-!IN50_LAUNCH!-!IN50_LAUNCH_PCT!-!IN50_MID_STOP!-!"
"IN50_MID_STOP_DIR!-!IN50_MID_STOP_DIR_RETURN!-!IN50_MID_STOP_RETURN!-!IN50_RPT!-!IN50_RPT_DIR!-!"
"IN50_RPT_DIR_RETURN!-!IN50_RPT_RETURN!-!IN50_TRANS_DIR!-!IN50_TRANS_DIR_RETURN!-!IN50_TRANS_RETURN!-!"
"IN50_TARGET!-!IN50_TARGET_CHANCE!-!IN50_TARGET_CHANCE_PCT!-!IN50_TARGET_DISTANCE!-!IN50_TARGET_PLAY_GOAL!-!"
"IN50_TARGET_PLAY_GOAL_PCT!-!IN50_TARGET_PLAY_RET_PCT!-!IN50_TARGET_PLAY_RET!-!IN50_TARGET_PLAY_SCORE!-!"
"IN50_TARGET_PLAY_SCORE_PCT!-!IN50_TARGET_TEAM_GOAL!-!IN50_TARGET_TEAM_GOAL_PCT!-!IN50_TARGET_TEAM_RET_PCT!-!"
"IN50_TARGET_TEAM_RET!-!IN50_TARGET_TEAM_SCORE!-!IN50_TARGET_TEAM_SCORE_PCT!-!IN50_TRANS")

# Function to fetch data with given parameters
def fetch_data(session, competition_id, season_id, stat_selection, rd="", base_url="https://recruit.championdata.com/report.jsp"):
    params = {
        'tab_id': 'ui-tabs-1',
        'contains_top8_opp_squad_id': 'true',
        'minGames': 'Min Rnds',
        'customLevel': '25',
        'level':'99999',
        'contains_all_sep': 'true',
        'msort_way': 'asc',
        'sort_way': 'asc',
        'requestMade': '1',
        'page_id': '98',
        'statSelection': '!-!'.join(stat_selection),
        'groupSelection': 'GROUP_ROUND_NO!--!SQUAD_ID', #'OPP_SQUAD_ID!--!, !--!PERIOD!, !--!ZONE_LOGICAL_SM
        'competition_level_id': competition_id,
        'competition_group_id': '1',
        'season_id': season_id,
        'round_no': rd,
        'minGames':'Min Rnds',
        'total_average': 'T',
        'show': 'for',
        'msort': '1',
        'isSplitMidfield': 'true',
        'ignore_club': '1',
        'isFeeder': 'true'
    }

    if competition_id == 1:
        del params['competition_level_id']
        params['isFeeder'] = 'false'
        base_url = "https://cia.championdata.com/report.jsp"
    
    # random sleep
    time.sleep(random.uniform(0.5, 4.5))
    
    response = session.get(base_url, params=params)
    
    if response.status_code == 200:
        return response.text
    else:
        return None

def parse_data(html_data):

    df = pd.read_html(io=StringIO(html_data), match='Dimensions', attrs={'class': 'mbl mbr mbt'}, header=1)[0]
    
    return df

In [4]:
# Parameters to parse
competition_ids = list(competitions_dict.keys())
season_ids = ["2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2022B","2023","2024"] 
all_stats = [list(filter(None, sublist.split("!-!"))) for sublist in dimensional_custom_selections.split("*")]
no_derivations = [[s for s in sublist if not any(substring in s for substring in ["PCT", "EFFICIENCY", "RATIO", "RATE"])] for sublist in all_stats]
rounds_batches = ['1-25'] #['1-5','6-10','11-15','16-20','20-25']

In [5]:
# e.g.

session = login_and_get_session()
competition_id = 1 #11
competition_name = "AFL" #"NAB League"
season_id = 2023
rounds = '1-25'
stat_batch = ['DISPOSAL',
 'EFFECTIVE_DISPOSAL',
 'EFFECTIVE_HANDBALL',
 'EFFECTIVE_KICK',
 'HANDBALL']

html_data = fetch_data(session, competition_id, season_id, stat_batch, rd=rounds)
df = parse_data(html_data)

df.columns = ['#', 'Player', 'Position', 'S', 'Mt', 'R'] + stat_batch
df['season_id'] = season_id
df['competition_id'] = competition_id
df['competition_name'] = competition_name

filename = f'./player_match_data/players_{competition_name}_{season_id}_{rounds}'
df.set_index(['Player', 'Position', 'S', 'Mt', 'R', 'season_id', 'competition_id']).to_csv(f"{filename}.csv")

Cookies after login:


In [9]:
# MAIN LOOP

session = login_and_get_session()

# random.shuffle(competition_ids)

for competition_id in competition_ids[1:]:
    
    competition_name = competitions_dict[competition_id]['Name']
    
    for season_id in season_ids[-3:]:

        season_data = []
        start = time.time()
        
        if (season_id == "2022B" and competition_id == 264) or (season_id.isdigit() and int(season_id) >= competitions_dict[competition_id]['Start']):
        
            start = time.time()

            print(f"Parsing {competition_name}, {season_id}")
            
            #for rounds in tqdm(rounds_batches, desc='Rounds'):
            for submenu in tqdm(no_derivations,leave=False, desc='Stat menu'):
                for i in tqdm(list(range(0, len(submenu), 5)),leave=False, desc='Stat batch'):
                    stat_batch = submenu[i:i+5]
                    html_data = fetch_data(session, competition_id, season_id, stat_batch)

                    if html_data:
                        try:
                            df = parse_data(html_data)
                            df['season_id'] = season_id
                            df['competition_id'] = competition_id
                            df['competition_name'] = competition_name
                            season_data.append(df)
                        except:
                            print(f"Failed to parse season {season_id} for competition {competition_name}, stats: {stat_batch}")
            
        else:
            continue

        print(f"Time taken for season {season_id}: {time.time() - start} seconds")
        
        # Concatenate all dataframes
        season_df = pd.concat([df.set_index(['#', 'Mt', 'Player', 'Position', 'S', 'R', 'season_id', 'competition_id', 'competition_name']) for df in season_data], axis=1)
            
        # Save to CSV
        filename = f'players_{competition_name}_{season_id}'
        season_df.to_csv(f'./scraped_data/{filename}.csv', index=True)
        print(f"Data scraped and saved to {filename}.csv")

Cookies after login:
Parsing AFL, 2023


Stat menu:   0%|          | 0/24 [00:00<?, ?it/s]

Stat batch:   0%|          | 0/3 [00:00<?, ?it/s]

Stat batch:   0%|          | 0/2 [00:00<?, ?it/s]

Stat batch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Stat batch:   0%|          | 0/1 [00:00<?, ?it/s]

Stat batch:   0%|          | 0/2 [00:00<?, ?it/s]

Stat batch:   0%|          | 0/3 [00:00<?, ?it/s]

Stat batch:   0%|          | 0/7 [00:00<?, ?it/s]

Time taken for season 2024: 2250.158361196518 seconds
Data scraped and saved to players_AFL_2024.csv


In [116]:
# THINGS TO TRY:
# DONE 1. Batch stats to max 5 per query MANY SMALL VS FEW LARGE (~ 10 mins per season)
# DONE 2. Get rid of pct and ratios, especially those that can be recreated from game totals
# 3. Consider including QbQ, Zonal granularity (esp to teams level). If these reports are too tall, these can be split as seperate calls by using the filters.
# DONE 4. parse round by round (less aggregation on their server required)
# 5. TODO: Ensure all players data

In [ ]:
# SAMPLE OUTPUT
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

final_df.head(45)

In [66]:
# Count NAs

def count_nas_percentage(df):
    na_counts = df.isna().sum()
    total_rows = len(df)
    na_percentage = (na_counts / total_rows) * 100
    return na_percentage

count_nas_percentage(final_df)

#              0.0
BMBMfD         0.0
BPfD %         0.0
CMBMfD         0.0
CPfD %         0.0
D50R-I50       0.0
#              0.0
D50R-I50P %    0.0
LDL-K          0.0
LDL-L          0.0
LDL-LP %       0.0
LDL-O          0.0
#              0.0
LDL-P %        0.0
LDL-R          0.0
LDL-RP %       0.0
LDL-S          0.0
LDL-SP %       0.0
#              0.0
LDL-S          0.0
LDL-SP %       0.0
MBMB           0.0
MBMBP %        0.0
MBMC           0.0
#              0.0
MBMCP %        0.0
MBMfD          0.0
MBMW           0.0
MBMWP %        0.0
WMBMfD         0.0
#              0.0
WPfD %         0.0
#              0.0
D              0.0
DE %           0.0
ED             0.0
EH             0.0
EK             0.0
#              0.0
H              0.0
HE %           0.0
K              0.0
KtHR           0.0
KE %           0.0
dtype: float64

In [80]:
# Count zeroes

def count_zeros_percentage(df):
    zero_counts = df.apply(lambda col: (col == 0).sum())
    total_rows = len(df)
    zero_percentage = (zero_counts / total_rows) * 100
    return zero_percentage

count_zeros_percentage(final_df)

#             0.00
BMBMfD      100.00
CMBMfD      100.00
D50R-I50      0.25
LDL-K       100.00
LDL-L       100.00
#             0.00
LDL-O       100.00
LDL-R       100.00
LDL-S       100.00
LDL-S.1     100.00
MBMB        100.00
#             0.00
MBMC        100.00
MBMfD         0.00
MBMW        100.00
WMBMfD      100.00
#             0.00
D             0.00
ED            0.00
EH            0.00
EK            0.00
H             0.00
#             0.00
K             0.00
#             0.00
D-P           0.00
D-P.1         0.00
#             0.00
5P           42.50
FA            0.00
FAG           0.00
FAM           2.00
FAR          24.25
#             0.00
FAT           0.00
FF            0.00
FF-P          0.00
FF-P.1        1.00
MPA          42.50
#             0.00
CK            0.00
EKS           0.75
ELK         100.00
GK            5.25
IK            0.00
#             0.00
KB            0.75
LEKFP       100.00
LKFP        100.00
OOF           3.50
SEKFP       100.00
#           